In [1]:
import sys
sys.path.append('/Users/simon/git/burns/code/')

In [2]:
import glob
input_set = glob.glob('/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/stain_files/*.mzML')
print input_set

['/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/stain_files/Nicotine_Stain_A.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/stain_files/Nicotine_Stain_B.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/stain_files/Nicotine_Stain_C.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/stain_files/Tea_Stain_A.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/stain_files/Tea_Stain_B.mzML', '/Users/simon/Dropbox/BioResearch/Meta_clustering/Burns/stain_files/Tea_Stain_C.mzML']


In [3]:
from load_di import load_di

determination of memory status not supported on this 
 platform, mesauring for memoryleaks will never fail


In [34]:
data = {}
for f in input_set:
    data[f] = load_di(f,normalise = 1000.0)

In [35]:
from matching import Greedy
g = Greedy()
mp = g.process(data,5)

2776 peaks


In [36]:
name_list = []
name_dict = {}
for f in input_set:
    short_name = f.split('/')[-1]
    name_list.append(f)
    name_dict[f] = short_name
name_list = sorted(name_list)

In [37]:
other = []
other_file = '/Users/simon/git/burns/notebooks/spreadsheets/both_v_both/both_v_both_log_1e-05_tus.csv'
with open(other_file,'r') as f:
    reader = csv.reader(f)
    heads = reader.next()
    for line in reader:
        other.append(line)
mz_list = [float(m[0]) for m in other]
print mz_list

[328.0816026655655, 327.0782204558983, 283.1177664655883, 349.0602078719774, 124.09703586735463, 284.12135756014277, 327.17593850563134, 305.0996424997124, 223.0966966238663, 207.06541002325974, 345.1080984823646, 327.0082160428099, 404.2200098980612, 344.1048564438347, 208.0687948897043, 294.20662851302734, 163.0391754265429, 273.02889781226355, 305.1281140004943, 152.94730577664998, 245.0786011340925, 227.16436113544952, 128.98613828133475, 432.23842101086035, 364.9641672708975, 520.1394045526376, 348.9902687881293, 258.0286347444263, 321.0737243422642, 202.0711236978032, 150.02686617118857, 177.0547917754836, 350.06361930521655, 286.1287329741182, 379.2522057595379, 300.14440977232005, 322.0769935969639, 224.10008676067653, 79.02147268736907, 328.17935362585155, 230.9904123252886, 291.08417619015677, 79.01811607573103, 240.02703903941108, 184.08396409809788, 434.24488714597624, 365.03415057091297, 236.90874508647292, 168.1232402704431, 269.1021895743783, 164.04249032338308, 307.0579

In [38]:
import csv
rows = []

for ps in mp:
    row = []
    row = [ps.mean_mz]
    mz = float(ps.mean_mz)
    deltas = [1e6*abs(mz - q)/mz for q in mz_list]
    matches = [(i,d) for i,d in enumerate(deltas)]
    matches = filter(lambda x: x[1] < 5,matches)
    if len(matches) == 0:
        row += ['no_match','nan']
    else:
        matches = sorted(matches,key = lambda x: x[1])
        row += [mz_list[matches[0][0]],matches[0][1]]
    for name in name_list:
        peaks = filter(lambda x: x[2] == name,ps.peaks)
        if len(peaks) == 0:
            row.append('nan')
        else:
            peaks = sorted(peaks,key = lambda x: x[1])
            row.append(peaks[0][1])
    rows.append(row)

rows = sorted(rows,key = lambda x: sum([1 if m=='nan' else 0 for m in x]))



with open('spreadsheets/stains/stains.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['mz'] + ['original_mz'] + ['ppm_error'] + [name_dict[s] for s in name_list])
    for row in rows:
        writer.writerow(row)
            